In [24]:
import pandas as pd

import DataRetriever as dr
import SARIMA_forecast
import FFT_forecast

RETRIEVER = dr.DataRetriever()
CON_ATTRIBUTES = RETRIEVER.get_attributes(file_name='consuming_attributes.pkl')
PV_ATTRIBUTES = RETRIEVER.get_attributes(file_name='producing_attributes.pkl')
FLEX_ATTRIBUTES = ["Load_ClothesWasherPowerWithStandby", "Elec_PowerDishwasher", "Load_DryerPowerTotal"]
FIXED_ATTRIBUTES = list(set(CON_ATTRIBUTES) - set(FLEX_ATTRIBUTES))

In [25]:
def rescheduler(year, month, day):
    date = pd.Timestamp(year=year, month=month, day=day)
    pv = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[PV_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000
    fixed = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[FIXED_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000

    flex_clothes = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Load_ClothesWasherPowerWithStandby"].clip(lower=0) / 1000
    flex_dish = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Elec_PowerDishwasher"].clip(lower=0) / 1000
    flex_dryer = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Load_DryerPowerTotal"].clip(lower=0) / 1000

    # 7 days of training data - subject to be changed, depends on fcast model
    pv = pv.loc[(pv.index >= date - pd.Timedelta(days=7)) & (pv.index < date)]
    fixed = fixed.loc[(fixed.index >= date - pd.Timedelta(days=28)) & (fixed.index < date)]

    flex_clothes = flex_clothes.loc[(flex_clothes.index >= date) & (flex_clothes.index < date + pd.Timedelta(days=3))]
    flex_dish = flex_dish.loc[(flex_dish.index >= date) & (flex_dish.index < date + pd.Timedelta(days=3))]
    flex_dryer = flex_dryer.loc[(flex_dryer.index >= date) & (flex_dryer.index < date + pd.Timedelta(days=3))]

    # ------------------ Forecast PV ------------------
    pv_pred = pd.DataFrame(data=FFT_forecast.fourierExtrapolation(data=pv, number_of_predictions=72, n_sinusoids=5)[len(pv):], index=flex_dish.index)

    # ------------------ Forecast Fixed ------------------
    fixed_pred = SARIMA_forecast.sarima_prediction(date, fixed)

    # Concat results
    df = pd.concat([pv_pred, fixed_pred, flex_clothes, flex_dish, flex_dryer], axis=1)

    df = df.T.reset_index(drop=True).T  # Easy way to remove column names
    df.rename(columns={0: "PV", 1: "FIXED", 2: "FLEX_CLOTHES", 3: "FLEX_DISH", 4: "FLEX_DRYER"}, inplace=True)

    # Stand-by values set to 0, simplifies the code
    df.loc[df["FLEX_CLOTHES"] <= 0.002947, "FLEX_CLOTHES"] = 0

    df["Balance"] = df["PV"] - (df["FIXED"] + df["FLEX_CLOTHES"] + df["FLEX_DISH"] + df["FLEX_DRYER"])

    # pd.DataFrames with flexible appliances turned on and surplus where at least one flexible is not turned on.
    deficit = df[(df["Balance"] < 0) & ((df["FLEX_DISH"] > 0) | (df["FLEX_DRYER"] > 0) | (df["FLEX_CLOTHES"] > 0))]
    surplus = df[(df["Balance"] > 0) & ((df["FLEX_DISH"] == 0) | (df["FLEX_DRYER"] == 0) | (df["FLEX_CLOTHES"] == 0))]

    columns_dict = {0: "FLEX_CLOTHES", 1: "FLEX_DISH", 2: "FLEX_DRYER"}
    res_df = pd.DataFrame(columns=["Planned Time", "Appliance", "Suggested Time"])

    # Loops over all rows in the deficit to find better times to turn on appliances.
    for idx, row in deficit.iterrows():
        while row[["FLEX_CLOTHES", "FLEX_DISH", "FLEX_DRYER"]].max() > 0: # Thus, a flexible appliance is active.
            col = columns_dict[row[["FLEX_CLOTHES", "FLEX_DISH", "FLEX_DRYER"]].argmax()] # Column of the largest deficit.

            deficit_value = deficit.at[idx, col] # Significance of deficit
            surplus_possible = surplus[(surplus["Balance"] >= deficit_value) & (surplus[col] == 0)] # Possible time slots

            if len(surplus_possible) == 0:  # If in a position where it is not possible to move, skip to next row
                break

            suggested_time = surplus_possible.index[surplus_possible.index.get_indexer([idx], method='nearest')][0] # Closest time with enough surplus

            surplus.at[suggested_time, col] = deficit_value
            surplus.at[suggested_time, "Balance"] -= deficit_value # Change balance in the surplus df

            deficit.at[idx, col] = 0 # Set the deficit value to 0

            res_df.loc[len(res_df)] = (idx, col, suggested_time) # Insert rows to final result

    return res_df

In [26]:
suggested_rescheduler = rescheduler(2016, 1, 18)
suggested_rescheduler

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


,Planned Time,Appliance,Suggested Time
0,2016-01-18 20:00:00,FLEX_DISH,2016-01-18 15:00:00
1,2016-01-18 21:00:00,FLEX_DISH,2016-01-18 14:00:00
2,2016-01-18 22:00:00,FLEX_DISH,2016-01-18 13:00:00
3,2016-01-20 18:00:00,FLEX_CLOTHES,2016-01-20 15:00:00
4,2016-01-20 19:00:00,FLEX_CLOTHES,2016-01-20 14:00:00
5,2016-01-20 20:00:00,FLEX_DRYER,2016-01-20 13:00:00
6,2016-01-20 20:00:00,FLEX_CLOTHES,2016-01-20 13:00:00
7,2016-01-20 20:00:00,FLEX_DISH,2016-01-20 15:00:00
8,2016-01-20 21:00:00,FLEX_DISH,2016-01-20 14:00:00
9,2016-01-20 21:00:00,FLEX_CLOTHES,2016-01-20 12:00:00


In [28]:
date = pd.Timestamp(year=2016, month=1, day=18)
pv = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[PV_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000
fixed = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')[FIXED_ATTRIBUTES].sum(axis=1).clip(lower=0) / 1000

flex_clothes = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Load_ClothesWasherPowerWithStandby"].clip(lower=0) / 1000
flex_dish = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Elec_PowerDishwasher"].clip(lower=0) / 1000
flex_dryer = RETRIEVER.get_data(file_name='All-Subsystems-hour-Year2.pkl')["Load_DryerPowerTotal"].clip(lower=0) / 1000

pv = pv.loc[(pv.index >= date) & (pv.index < date + pd.Timedelta(days=3))]
fixed = fixed.loc[(fixed.index >= date) & (fixed.index < date + pd.Timedelta(days=3))]

flex_clothes = flex_clothes.loc[(flex_clothes.index >= date) & (flex_clothes.index < date + pd.Timedelta(days=3))]
flex_dish = flex_dish.loc[(flex_dish.index >= date) & (flex_dish.index < date + pd.Timedelta(days=3))]
flex_dryer = flex_dryer.loc[(flex_dryer.index >= date) & (flex_dryer.index < date + pd.Timedelta(days=3))]

df = pd.concat([pv, fixed, flex_clothes, flex_dish, flex_dryer], axis=1)
# Easy way to remove column names
df = df.T.reset_index(drop=True).T
df.rename(columns={0: "PV", 1: "FIXED", 2: "FLEX_CLOTHES", 3: "FLEX_DISH", 4: "FLEX_DRYER"}, inplace=True)

# Stand-by values set to 0, simplifies the code
df.loc[df["FLEX_CLOTHES"] <= 0.002947, "FLEX_CLOTHES"] = 0

df["Balance"] = df["PV"] - (df["FIXED"] + df["FLEX_CLOTHES"] + df["FLEX_DISH"] + df["FLEX_DRYER"])
df

,PV,FIXED,FLEX_CLOTHES,FLEX_DISH,FLEX_DRYER,Balance
Timestamp,,,,,,
2016-01-18 00:00:00,0.018460,1.534748,0.000000,0.000000,0.000000,-1.516289
2016-01-18 01:00:00,0.018366,1.694601,0.000000,0.000000,0.000000,-1.676235
2016-01-18 02:00:00,0.018294,1.820077,0.000000,0.000000,0.000000,-1.801782
2016-01-18 03:00:00,0.018351,1.614793,0.000000,0.000000,0.000000,-1.596442
2016-01-18 04:00:00,0.018398,1.747775,0.000000,0.000000,0.000000,-1.729377
...,...,...,...,...,...,...
2016-01-20 19:00:00,0.017997,2.318495,0.080852,0.000000,0.000000,-2.381349
2016-01-20 20:00:00,0.018141,2.970168,0.100377,0.006675,1.696585,-4.755664
2016-01-20 21:00:00,0.018224,3.180637,0.068936,0.359280,0.000000,-3.590630


In [30]:
balance_pre = sum(df["Balance"][df["Balance"] < 0])
balance_pre

-118.82327744458001

-118.82327744458001

In [31]:
suggested_rescheduler

,Planned Time,Appliance,Suggested Time
0,2016-01-18 20:00:00,FLEX_DISH,2016-01-18 15:00:00
1,2016-01-18 21:00:00,FLEX_DISH,2016-01-18 14:00:00
2,2016-01-18 22:00:00,FLEX_DISH,2016-01-18 13:00:00
3,2016-01-20 18:00:00,FLEX_CLOTHES,2016-01-20 15:00:00
4,2016-01-20 19:00:00,FLEX_CLOTHES,2016-01-20 14:00:00
5,2016-01-20 20:00:00,FLEX_DRYER,2016-01-20 13:00:00
6,2016-01-20 20:00:00,FLEX_CLOTHES,2016-01-20 13:00:00
7,2016-01-20 20:00:00,FLEX_DISH,2016-01-20 15:00:00
8,2016-01-20 21:00:00,FLEX_DISH,2016-01-20 14:00:00
9,2016-01-20 21:00:00,FLEX_CLOTHES,2016-01-20 12:00:00


In [32]:
for idx, row in suggested_rescheduler.iterrows():
    df.at[row["Suggested Time"], row["Appliance"]] = df.at[row["Planned Time"], row["Appliance"]]
    df.at[row["Planned Time"], row["Appliance"]] = 0

In [33]:
df["New_Balance"] = df["PV"] - (df["FIXED"] + df["FLEX_CLOTHES"] + df["FLEX_DISH"] + df["FLEX_DRYER"])
df

,PV,FIXED,FLEX_CLOTHES,FLEX_DISH,FLEX_DRYER,Balance,New_Balance
Timestamp,,,,,,,
2016-01-18 00:00:00,0.018460,1.534748,0.0,0.0,0.0,-1.516289,-1.516289
2016-01-18 01:00:00,0.018366,1.694601,0.0,0.0,0.0,-1.676235,-1.676235
2016-01-18 02:00:00,0.018294,1.820077,0.0,0.0,0.0,-1.801782,-1.801782
2016-01-18 03:00:00,0.018351,1.614793,0.0,0.0,0.0,-1.596442,-1.596442
2016-01-18 04:00:00,0.018398,1.747775,0.0,0.0,0.0,-1.729377,-1.729377
...,...,...,...,...,...,...,...
2016-01-20 19:00:00,0.017997,2.318495,0.0,0.0,0.0,-2.381349,-2.300498
2016-01-20 20:00:00,0.018141,2.970168,0.0,0.0,0.0,-4.755664,-2.952027
2016-01-20 21:00:00,0.018224,3.180637,0.0,0.0,0.0,-3.590630,-3.162414


In [34]:
balance_post = sum(df["New_Balance"][df["New_Balance"] < 0])
balance_post

-117.520288194811

In [37]:
(1 - balance_post / balance_pre) * 100

1.0965774365016445